In [2]:
!pip -q install langchain openai tiktoken PyPDF2 faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

# Chat & Query your PDF files

In [3]:
#!pip install langchain_community
!pip install -q sentence-transformers
!pip install -q langchain


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 4.6 MB/s eta 0:00:00


In [5]:
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00


### Basic Chat PDF


In [6]:
from PyPDF2 import PdfReader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

## Reading in the PDF


In [7]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [10]:
from PyPDF2 import PdfReader

# Replace 'impromptu-rh.pdf' with the correct path if needed
pdf_path = '/content/drive/My Drive/impromptu-rh.pdf'  # Adjust if it's in a subfolder
doc_reader = PdfReader(pdf_path)


In [11]:
doc_reader

In [12]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(doc_reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [13]:
len(raw_text)

371090

In [14]:
raw_text[:100]

'Impromptu\nAmplifying Our Humanity \nThrough AI\nBy Reid Hoffman  \nwith GPT-4Impromptu: AmplIfyIng our '

### Text Splitter

This takes the text and splits it into chunks. The chunk size is characters not tokens

In [15]:
# Text splitting
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 500,  # Reduce chunk size to 500
    chunk_overlap  = 100,  # Slightly reduce overlap
    length_function = len,
)
texts = text_splitter.split_text(raw_text)


In [16]:
len(texts)

913

In [17]:
texts[20]

'one. They’re just going to stare at you with that smug \nlook on their face and say, “You know this is a serious \nsafety hazard. You could have a fire, you could have a \nlawsuit, you could have a bad Yelp review. You better get \non that, pal.” And then they leave, and you’re left with a \ndark corner of your dining room, where no one wants to \nsit, where the food looks unappetizing, where the mood \nis gloomy. And you think to yourself, how many restau-'

In [18]:
texts[10]

'zation founded in 2015 with a mission to give millions of people \ndirect, hands-on access to powerful new AI tools.2Impromptu: Amplifying Our Humanity Through AI\nAs one of OpenAI’s original funders, I’ve been experimenting \nwith its products for some time now, so my query to GPT-4 was \nnot the first time I’d asked an LLM to create a lightbulb joke \nfor me. (I’m fond of lightbulb jokes.) While earlier versions of \nGPT sometimes understood the assignment—especially if the'

## Making the embeddings

In [19]:
# Use a smaller and faster model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")


<ipython-input-19-992ddb24b03e>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, 

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
import time

In [22]:
# Start the timer
start_time = time.time()

docsearch = FAISS.from_texts(texts, embedding_model)


# End the timer
end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time

# Print the time taken in seconds
print(f"Time taken to create FAISS index: {time_taken:.2f} seconds")

Time taken to create FAISS index: 1.45 seconds


In [23]:
docsearch.embedding_function

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
), model_name='sentence-transformers/paraphrase-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [24]:
query = "how does GPT-4 change social media?"
docs = docsearch.similarity_search(query)

In [25]:
len(docs)

4

In [27]:
docs[0]

Document(metadata={}, page_content='rected ways that tools like GPT-4 and DALL-E 2 enable.\nThis is a theme I’ve touched on throughout this travelog, but \nit’s especially relevant in this chapter. From its inception, social \nmedia worked to recast broadcast media’s monolithic and \npassive audiences as interactive, democratic communities, in \nwhich newly empowered participants could connect directly \nwith each other. They could project their own voices broadly, \nwith no editorial “gatekeeping” beyond a given platform’s terms')

**The above similarity Search result we, I will pass to language model**

## Plain QA Chain

In [31]:
!pip install -q langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 6.0 MB/s eta 0:00:00


In [40]:
from langchain_groq import ChatGroq
from langchain.chains.question_answering import load_qa_chain


In [34]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = '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',
    model="llama-3.1-70b-versatile"

)



In [49]:
# Pass the `llm` instance to the `load_qa_chain` function
chain = load_qa_chain(llm, chain_type="stuff")
 # we are going to stuff all the docs in at once

In [50]:
for message in chain.llm_chain.prompt.messages:
    print(message)


prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}") additional_kwargs={}
prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}') additional_kwargs={}


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:

In [51]:
query = "who are the authors of the book?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

'The text does not explicitly state the authors of the book, but it mentions that Reid Hoffman invited someone to co-author the book with him. However, the name of the co-author is not provided in the given text.'

In [52]:
query = "who is the author of the book?"
query_02 = "has it rained this week?"
docs = docsearch.similarity_search(query_02)
chain.run(input_documents=docs, question=query)

"I don't know the author of the book. The provided text does not mention the author's name."

In [53]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=4)
chain.run(input_documents=docs, question=query)

"The text doesn't explicitly state the name of the author of the book. However, it mentions that the author is co-authoring the book with Reid Hoffman."

In [56]:
chain = load_qa_chain(llm,
                      chain_type="stuff") # we are going to stuff all the docs in at once

In [57]:
query = "who is the book authored by?"
docs = docsearch.similarity_search(query,k=20)   # Retrieves up to 20 relevant documents
chain.run(input_documents=docs, question=query)  # retrieval-enhanced stuffing

'The book "Impromptu: Amplifying Our Humanity Through AI" is authored by Reid Hoffman, with contributions from GPT-4.'

### QA Chain with map-reduce method

In [64]:
chain = load_qa_chain(llm,
                      chain_type="map_rerank",
                      return_intermediate_steps=True # it will return intermmediate steps for each on of these it is calling language model
                      )

query = "who are openai?"
docs = docsearch.similarity_search(query,k=10)
results = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
results

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


{'intermediate_steps': [{'answer': 'Helpful Answer: OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity.',
   'score': '100'},
  {'answer': 'Helpful Answer: OpenAI is an organization that aims to develop technologies that put the power of AI directly into the hands of millions of people, functioning as a decentralized, personally empowering force.',
   'score': '80'},
  {'answer': "Helpful Answer: I don't know", 'score': '0'},
  {'answer': "Helpful Answer: I don't know, but based on the context, OpenAI seems to be an organization that has developed the GPT-4 model and has team members such as Sam, Greg, and Mira.",
   'score': '20'},
  {'answer': "Helpful Answer: I don't know", 'score': '0'},
  {'answer': 'Helpful Answer: OpenAI is an organization that develops and releases AI tools, such as DALL-E 2 and ChatGPT, with the mission of giving millions of users hands-on access to these tools.',
   'score': '80'},
  {'answer':

In [65]:
results['output_text'] #THis is final output tex

'Helpful Answer: OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity.'

In [ ]:
results['intermediate_steps']

[{'answer': ' OpenAI is an organization that released text-to-image generation tool DALL-E 2 and ChatGPT in April 2022 and are giving millions of users hands-on access to these AI tools.',
  'score': '80'},
 {'answer': ' OpenAI is a research laboratory whose founding goal is to develop technologies that put the power of AI directly into the hands of millions of people.',
  'score': '80'},
 {'answer': ' OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity.',
  'score': '100'},
 {'answer': ' OpenAI is a technology company focused on using artificial intelligence to solve real-world problems.',
  'score': '80'},
 {'answer': ' OpenAI is a company co-founded by Sam Altman that is developing AI technologies and allowing individuals to participate in the development process.',
  'score': '90'},
 {'answer': ' OpenAI is a research laboratory that focuses on artificial intelligence technologies.',
  'score': '80'},
 {'answer': ' Ope

In [ ]:
# check the prompt
chain.llm_chain.prompt.template

"Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nIn addition to giving an answer, also return a score of how fully it answered the user's question. This should be in the following format:\n\nQuestion: [question here]\nHelpful Answer: [answer here]\nScore: [score between 0 and 100]\n\nHow to determine the score:\n- Higher is a better answer\n- Better responds fully to the asked question, with sufficient level of detail\n- If you do not know the answer based on the context, that should be a score of 0\n- Don't be overconfident!\n\nExample #1\n\nContext:\n---------\nApples are red\n---------\nQuestion: what color are apples?\nHelpful Answer: red\nScore: 100\n\nExample #2\n\nContext:\n---------\nit was night and the witness forgot his glasses. he was not sure if it was a sports car or an suv\n---------\nQuestion: what type was the car?\nHelpful Answer: a sports car or an su

## RetrievalQA Chain
RetrievalQA chain uses load_qa_chain and combines it with the a retriever (in our case the FAISS index)

In [67]:
from langchain.chains import RetrievalQA

# set up FAISS as a generic retriever
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":4})

# create the chain to answer questions
rqa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

In [70]:
rqa("What is OpenAI?")

{'query': 'What is OpenAI?',
 'result': 'OpenAI is a research organization that develops and shares artificial intelligence tools for the benefit of humanity.',
 'source_documents': [Document(metadata={}, page_content='words or less) (make prose more lively and less wooden \nthan usual)\nGPT-4:  The short answer is: no. OpenAI is a research \norganization that develops and shares artificial intelli-\ngence tools for the benefit of humanity. It does not claim \nany ownership or rights over the content that its tools \nproduce or help produce. You are free to use the GPT \ntool as a source of inspiration, but you are responsible for \nyour own originality, quality, and legality of your work.'),
  Document(metadata={}, page_content='ommendations, and more.\nA founding goal of OpenAI was to develop technologies that put \nthe power of AI directly into the hands of millions of people. \nIn this way, AI might function as a decentralized, personally \nempowering force, rather than a top-down,

In [73]:
query = "Who is author of the book?"
rqa(query)['result']

'The text does not explicitly mention the author of the book. However, it does mention that Reid Hoffman co-authored the book with someone, but the name of the co-author is not provided.'

In [74]:
query = "What does gpt-4 mean for creativity?"
rqa(query)['result']

'According to the provided context, GPT-4 has the potential to aid and augment creativity, particularly for artists and creators. It can generate new ideas, add details, and provide suggestions that might not have been considered before. The text mentions that GPT-4 can "infer" what the user might want to know and provide additional information, which can be seen as a form of creative collaboration.\n\nHowever, it\'s also worth noting that the text does not suggest that GPT-4 replaces human creativity, but rather provides a new tool for generating new ideas and inputs that can be used to enhance and inform human creativity.'

In [75]:
query = "what have the last 20 years been like for American journalism?"
rqa(query)['result']

'According to the provided context, the last 20 years have been mostly bad news for the American journalism industry. The rise of the internet has led to increased competition for ad dollars from non-news players, which has had a negative impact on the industry.'

In [76]:
query = "how can journalists use GPT-4??"
rqa(query)['result']

"According to the provided context, journalists can use GPT-4 in the following ways:\n\n1. Familiarize themselves with AI tools like GPT-4: Journalists should learn about GPT-4 and its capabilities to stay relevant in the industry.\n2. Experiment with new ways to find and tell stories: GPT-4 can help journalists discover new angles and approaches to storytelling.\n3. Use GPT-4 to analyze data and generate summaries: For example, a journalist could ask GPT-4 to provide a 100-word summary of a news story or to analyze data on a particular topic.\n\nAdditionally, the context suggests that GPT-4 can be used to:\n\n* Generate automated journalism, such as producing thousands of stories about corporate earnings reports, weather forecasts, sports events, and more.\n* Assist journalists in researching and writing stories, potentially freeing up time for more in-depth reporting and analysis.\n\nHowever, it's also emphasized that a human touch is still necessary in journalism, and GPT-4 should b

In [77]:
query = "How is GPT-4 different from other models?"
rqa(query)['result']

'GPT-4 is different from its predecessors in that it has reached a new level of proficiency. It is skilled at generating prose of all kinds, including emails, poetry, essays, and more. It is also great at summarizing documents, has improved in translating languages, and writing computer code. Additionally, it can synthesize information from a wide range of sources extremely quickly, making it more efficient than searching through multiple links or sources.'

In [78]:
query = "What is beagle Bard?"
rqa(query)['result']

"I don't know what Beagle Bard is."